In [113]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py
import pandas as pd
from collections import OrderedDict
from pprint import pprint

%matplotlib notebook

In [2]:
import qcodes as qc

# Setup notes

For this example to run, you need to have set up a labpython config file. There is a template in labpythonQDelft/templates.

In [7]:
from pysweep.data_storage import base_storage
reload(base_storage)
from pysweep.data_storage.base_storage import BaseStorage, DataSet

from pysweep import sweep_object
reload(sweep_object)
from pysweep.sweep_object import sweep, ChainSweep

In [87]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement, PysweepMeasurement
from experiment.measurement import Data as _Data

In [88]:
station = qc.Station()

In [116]:
class Data(_Data):
    
    @staticmethod
    def get_column_names(page):
        colnames = [n for n in page.dtype.fields]
        return colnames
    
    @staticmethod
    def get_axes_names(page):
        n = Data.get_column_names(page)
        return n[:-1]


class GridData(Data):
    
    def __init__(self, filepath):
        super().__init__(filepath)
        self.grids = {}
        
    def add(self, record):
        super().add(record)
        
        independents = [(name, value) for name, value in record.items() if value.get('independent_parameter', False)]
        independents = self.dict_to_array(dict(independents))
        dependents = [(name, value) for name, value in record.items() if not value.get('independent_parameter', False)]
        
        for (n, v) in dependents:
            if n in self.grids:
                g = self.grids[n]
                if 'shape' not in g:
                    shp = tuple([a.size for _, a in g['axes'].items()])
                    g['shape'] = shp
        
    
    
ivals = np.arange(5, dtype=int)
jvals = np.arange(5) * 0.1
scales = np.linspace(0,1,3)

d = GridData(None)
d.grids['value'] = {}
d.grids['value']['axes'] = OrderedDict({
    'scale' : scales,
    'j' : jvals,
    'i' : ivals,
})

for i in ivals:
    for j in jvals:
        v = (i + j**2) * scales
        rec = OrderedDict({
            'scale' : {'value' : scales, 'independent_parameter' : True},
            'j' : {'value' : j, 'independent_parameter' : True},
            'i' : {'value' : i, 'independent_parameter' : True},
            'value' : {'value' : v},
        })
        d.add(rec)

In [117]:
pprint(d.grids)

{'value': {'axes': OrderedDict([('scale', array([ 0. ,  0.5,  1. ])),
                                ('j', array([ 0. ,  0.1,  0.2,  0.3,  0.4])),
                                ('i', array([0, 1, 2, 3, 4]))]),
           'shape': (3, 5, 5)}}
